In [1]:
!pip install kaggle

In [2]:
import os

os.environ["KAGGLE_USERNAME"] = "mohanra01"
os.environ["KAGGLE_KEY"] = "e3f2939076ed3f324f447d2e23b3453d"


In [3]:
!kaggle datasets download Cornell-University/arxiv

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
 99% 1.40G/1.41G [00:19<00:00, 54.7MB/s]
100% 1.41G/1.41G [00:19<00:00, 79.3MB/s]


In [4]:
!unzip /content/arxiv.zip


Archive:  /content/arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


In [5]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
import pandas as pd
import json
import re
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import joblib


In [7]:
def preprocess_text(text):
    """Text cleaning and preprocessing"""
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.split()
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    text = [ps.stem(word) for word in text if word not in stop_words]
    return ' '.join(text)

In [8]:
file_path = "/content/arxiv-metadata-oai-snapshot.json"
data = []

with open(file_path, "r") as f:
    for _ in range(10000):
      line = f.readline()
      if not line:
          break
      data.append(json.loads(line))

df = pd.DataFrame(data)

df["text"] = df["title"] + " " + df["abstract"]
df['clean_text'] = df['abstract'].fillna("").apply(preprocess_text)


In [9]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X = tfidf_vectorizer.fit_transform(df["clean_text"])
y = df["categories"]


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)


SVC(kernel='linear', probability=True)

In [11]:
y_pred = svm_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.447
Classification Report:
                                                                                          precision    recall  f1-score   support

                                                                               astro-ph       0.56      0.99      0.71       338
                                                                         astro-ph gr-qc       0.00      0.00      0.00         6
                                                                  astro-ph gr-qc hep-ph       0.00      0.00      0.00         1
                                                           astro-ph gr-qc hep-ph hep-th       0.00      0.00      0.00         5
                                                          astro-ph gr-qc hep-ph nucl-th       0.00      0.00      0.00         1
                                                                  astro-ph gr-qc hep-th       0.00      0.00      0.00         2
                                                        

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_

In [12]:
title = input("Enter Paper Title: ")
abstract = input("Enter Paper Abstract: ")
user_input = preprocess_text(title + " " + abstract)
X_input = tfidf_vectorizer.transform([user_input])


predicted_category = svm_model.predict(X_input)[0]
print(f"\n🔍 Predicted Paper Category: {predicted_category}")


Enter Paper Title: Lifetime of doubly charmed baryons
Enter Paper Abstract: In this work, we evaluate the lifetimes of the doubly charmed baryons\n$\\Xi_{cc}^{+}$, $\\Xi_{cc}^{++}$ and $\\Omega_{cc}^{+}$. We carefully calculate\nthe non-spectator contributions at the quark level where the Cabibbo-suppressed\ndiagrams are also included. The hadronic matrix elements are evaluated in the\nsimple non-relativistic harmonic oscillator model. Our numerical results are\ngenerally consistent with that obtained by other authors who used the diquark\nmodel. However, all the theoretical predictions on the lifetimes are one order\nlarger than the upper limit set by the recent SELEX measurement. This\ndiscrepancy would be clarified by the future experiment, if more accurate\nexperiment still confirms the value of the SELEX collaboration, there must be\nsome unknown mechanism to be explored.\n

🔍 Predicted Paper Category: hep-ph


In [ ]:
# "title":"Lifetime of doubly charmed baryons"
# "abstract":"  In this work, we evaluate the lifetimes of the doubly charmed baryons\n$\\Xi_{cc}^{+}$, $\\Xi_{cc}^{++}$ and $\\Omega_{cc}^{+}$. We carefully calculate\nthe non-spectator contributions at the quark level where the Cabibbo-suppressed\ndiagrams are also included. The hadronic matrix elements are evaluated in the\nsimple non-relativistic harmonic oscillator model. Our numerical results are\ngenerally consistent with that obtained by other authors who used the diquark\nmodel. However, all the theoretical predictions on the lifetimes are one order\nlarger than the upper limit set by the recent SELEX measurement. This\ndiscrepancy would be clarified by the future experiment, if more accurate\nexperiment still confirms the value of the SELEX collaboration, there must be\nsome unknown mechanism to be explored.\n","versions":[{"version":"v1","created":"Sat, 31 Mar 2007 07:04:26 GMT"}],"update_date":"2008-12-18","authors_parsed":[["Chang","Chao-Hsi",""],["Li","Tong",""],["Li","Xue-Qian",""],["Wang","Yu-Ming",""]]}
